Author : https://github.com/amitrm/

#### Import Packges

In [348]:
import os
import folium as fl
import pandas as pd
from folium.plugins import MarkerCluster
import datetime
import numpy as np
from IPython.display import clear_output
import matplotlib.pyplot as plt
from collections import defaultdict, deque

#### Import Datasets

In [349]:
stops = pd.read_csv('../gtfs-files/stops.txt')

stop_times = pd.read_csv('../gtfs-files/stop_times.txt')
stop_times = stop_times.sort_values(['trip_id', 'stop_sequence'], ascending = [1,1]).reset_index(drop = True)


In [350]:
stops.head(10)

,stop_id,stop_code,stop_name,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
0,1,NaN,Main St and Travelers,41.765179,-72.673435,NaN,NaN,0,NaN
1,10,NaN,Capitol Ave and Broad St,41.763621,-72.686946,NaN,NaN,0,NaN
2,100,NaN,Farmington Ave and Prospect Ave,41.765535,-72.715800,NaN,NaN,0,NaN
3,1000,NaN,Oak St and Farnham Dr,41.748698,-72.576505,NaN,NaN,0,NaN
4,10001,NaN,Corbin Ave and Governor St,41.689974,-72.798441,NaN,NaN,0,NaN
5,10003,NaN,Corbin Ave and Opp Walker Rd,41.692199,-72.796431,NaN,NaN,0,NaN
6,10004,NaN,Skipper St and Helen Dr,41.696274,-72.797858,NaN,NaN,0,NaN
7,10005,NaN,Skipper St and Scarlett Dr,41.698115,-72.798414,NaN,NaN,0,NaN
8,10006,NaN,Burritt St and 475 Burritt St,41.677292,-72.793740,NaN,NaN,0,NaN
9,10007,NaN,Osgood Ave and Lawrence St,41.684397,-72.796568,NaN,NaN,0,NaN


In [351]:
stop_times.head(10)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,1074825,16:20:00,16:20:00,13103,1,NaN,0,0,0.0000
1,1074825,16:20:39,16:20:39,13145,2,NaN,0,0,0.2708
2,1074825,16:21:04,16:21:04,13119,3,NaN,0,0,0.4446
3,1074825,16:21:39,16:21:39,13120,4,NaN,0,0,0.6872
4,1074825,16:22:08,16:22:08,13121,5,NaN,0,0,0.8868
5,1074825,16:22:58,16:22:58,13122,6,NaN,0,0,1.2330
6,1074825,16:24:00,16:24:00,13123,7,NaN,0,0,1.6313
7,1074825,16:24:25,16:24:25,13124,8,NaN,0,0,1.9059
8,1074825,16:24:59,16:24:59,13125,9,NaN,0,0,2.2761
9,1074825,16:25:17,16:25:17,13126,10,NaN,0,0,2.4728


#### Create a dataframe containing stop to stop travel time and distance 

In [352]:
trip_list = list(stop_times.trip_id.unique())

In [353]:
data = pd.DataFrame(columns = ['trip_id', 'trip_seg', 'fstop', 'tstop', 'time', 'dist'])

counter = 0

for trip in trip_list:
    
    clear_output(wait = True)
    
    trip_df = stop_times[stop_times.trip_id == trip]
    trip_id_list = [trip] * (trip_df.shape[0] - 1)
    trip_seg_list = list(trip_df.stop_sequence.values)[:-1]
    fstop_list = list(trip_df.stop_id.values)[:-1]
    tstop_list = list(trip_df.stop_id.values)[1:]

    fstop_ts_list = list(trip_df.arrival_time.str.replace(' ','0').values)[:-1]
    tstop_ts_list = list(trip_df.arrival_time.str.replace(' ','0').values)[1:]

    fstop_dist = list(trip_df.shape_dist_traveled.values)[:-1]
    tstop_dist = list(trip_df.shape_dist_traveled.values)[1:]
    
    new_trip_df = pd.DataFrame({'trip_id' : trip_id_list,
                                'trip_seg' : trip_seg_list,
                                'fstop' : fstop_list, 
                                'tstop' : tstop_list,
                                'fstop_ts' : fstop_ts_list,
                                'tstop_ts' : tstop_ts_list,
                                'fstop_dist' : fstop_dist,
                                'tstop_dist' : tstop_dist})
    
    new_trip_df['fstop_ts_hr'] = new_trip_df.fstop_ts.str[:2].astype(int)
    new_trip_df['fstop_ts_min'] = new_trip_df.fstop_ts.str[3:5].astype(int)
    new_trip_df['fstop_ts_sec'] = new_trip_df.fstop_ts.str[6:8].astype(int)
    new_trip_df['fstop_ts'] = new_trip_df['fstop_ts_hr'] * 3600 + new_trip_df['fstop_ts_min'] * 60 + new_trip_df['fstop_ts_sec']
    
    new_trip_df['tstop_ts_hr'] = new_trip_df.tstop_ts.str[:2].astype(int)
    new_trip_df['tstop_ts_min'] = new_trip_df.tstop_ts.str[3:5].astype(int)
    new_trip_df['tstop_ts_sec'] = new_trip_df.tstop_ts.str[6:8].astype(int)
    new_trip_df['tstop_ts'] = new_trip_df['tstop_ts_hr'] * 3600 + new_trip_df['tstop_ts_min'] * 60 + new_trip_df['tstop_ts_sec']
    
    new_trip_df['time'] = (new_trip_df['tstop_ts'] - new_trip_df['fstop_ts'])
    new_trip_df['dist'] = (new_trip_df['tstop_dist'] - new_trip_df['fstop_dist'])
   
    data = data.append(new_trip_df)
    
    counter = counter + 1
    
    print ('Progress : {}%'.format(np.round(counter*100/len(trip_list), 2)))
    print ('Processing : {} of {} trips'.format(counter, len(trip_list)))
    
data.trip_id = data.trip_id.astype(int)
data.trip_seg = data.trip_seg.astype(int)
data.time = data.time.astype(int)
data.dist = data.dist.astype(float)
data['stop_dummy'] = data.fstop.astype(str) + '-' + data.tstop.astype(str)

data = data.sort_values(['time'], ascending = [1]).drop_duplicates(['stop_dummy'], keep = 'first').drop(columns = ['fstop_ts', 'tstop_ts', 'fstop_dist', 'tstop_dist', 'stop_dummy'])
data = data.sort_values(['trip_id', 'trip_seg'], ascending = [1,1]).reset_index(drop = True)

data = data[['trip_id', 'trip_seg', 'fstop', 'tstop', 'time', 'dist']]

Progress : 100%
Processing : 17102 of 17102 trips


In [354]:
data.to_csv('../data/dijkstra_input.csv', index = False)

In [355]:
data.head(10)

,trip_id,trip_seg,fstop,tstop,time,dist
0,1074825,1,13103,13145,39,0.2708
1,1074825,2,13145,13119,25,0.1738
2,1074825,3,13119,13120,35,0.2426
3,1074825,4,13120,13121,29,0.1996
4,1074825,5,13121,13122,50,0.3462
5,1074825,6,13122,13123,62,0.3983
6,1074825,7,13123,13124,25,0.2746
7,1074825,8,13124,13125,34,0.3702
8,1074825,9,13125,13126,18,0.1967
9,1074825,10,13126,13127,46,0.4968


#### Dijkstra Module

The following implementation of Dijksta Algorithm can be found here : https://gist.github.com/mdsrosa/c71339cb23bc51e711d8

In [356]:
class Graph(object):
    def __init__(self):
        self.nodes = set()
        self.edges = defaultdict(list)
        self.distances = {}

    def add_node(self, value):
        self.nodes.add(value)

    def add_edge(self, from_node, to_node, distance):
        self.edges[from_node].append(to_node)
        self.edges[to_node].append(from_node)
        self.distances[(from_node, to_node)] = distance

def dijkstra(graph, initial):
    visited = {initial: 0}
    path = {}

    nodes = set(graph.nodes)

    while nodes:
        min_node = None
        for node in nodes:
            if node in visited:
                if min_node is None:
                    min_node = node
                elif visited[node] < visited[min_node]:
                    min_node = node
        if min_node is None:
            break

        nodes.remove(min_node)
        current_weight = visited[min_node]

        for edge in graph.edges[min_node]:
            try:
                weight = current_weight + graph.distances[(min_node, edge)]
            except:
                continue
            if edge not in visited or weight < visited[edge]:
                visited[edge] = weight
                path[edge] = min_node

    return visited, path

def shortest_path(graph, origin, destination):
    visited, paths = dijkstra(graph, origin)
    full_path = deque()
    _destination = paths[destination]

    while _destination != origin:
        full_path.appendleft(_destination)
        _destination = paths[_destination]

    full_path.appendleft(origin)
    full_path.append(destination)

    return visited[destination], list(full_path)

We get the list of all stops in the network. 

In [357]:
sid = list(set(data.fstop.unique().astype(str)) | set(data.tstop.unique().astype(str)))

#### Building the network

In [358]:
if __name__ == '__main__':
    graph = Graph()
    
    counter = 0
    for stop in sid:
        clear_output(wait = True)
        graph.add_node(stop)
        counter = counter + 1
        print ('Stop : {} | {} of {}'.format(stop, counter, len(sid)))
        print (type(stop))
        
    for index, row in data.iterrows():
        clear_output(wait = True)
        graph.add_edge(str(row[2]), str(row[3]), row[4])
        print (str(row[2]), str(row[3]), row[4])
        print('Segment : {} of {}'.format(index + 1, data.shape[0]))

('1056', '1058', 49)
Segment : 11529 of 11529


#### Find the shortest route

In [359]:
def find_shortest_route(fstop, tstop):
    result = shortest_path(graph, fstop, tstop)
    stops_df = pd.merge(pd.DataFrame({'stop_id' : [int(i) for i in result[1]], 'stop_seq' : range(1, len(result[1]) + 1)}), stops[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']], on = 'stop_id')
    dist_df = pd.merge(pd.DataFrame({'fstop' : result[1][:-1], 'tstop' : result[1][1:]}).astype('int64'), data[['fstop', 'tstop', 'time', 'dist']].astype(float), on = ['fstop', 'tstop'])
    
    print ('Travel time     : {} ({} seconds)'.format(str(datetime.timedelta(seconds = result[0])), result[0]))
    print ('Travel distance : {} miles'.format(dist_df.dist.sum()))
    print (' ')
    print (stops_df[['stop_id', 'stop_seq', 'stop_name']])
    return stops_df, dist_df

In [360]:
stops_df, dist_df = find_shortest_route('1310', '1170')

Travel time     : 0:08:25 (505 seconds)
Travel distance : 4.5514 miles
 
    stop_id  stop_seq                                     stop_name
0      1310         1                  Tower Ave and Opp Lebanon St
1      1311         2                     Tower Ave and Coventry St
2      7996         3           Coventry St and Burgdorf Health Ctr
3      1140         4                   Coventry St and Branford St
4      1141         5  Coventry St and Capitol Region Mental Health
5      1142         6        Vine St and City Of Htfd Human Service
6      1143         7             Vine St and Opp Mental Health Ctr
7      1144         8                       Vine St and Opp Love Ln
8      1145         9                   Vine St and Opp Westland St
9      1146        10                 Vine St and Opp Winchester St
10     1147        11                  Vine St and Opp Rockville St
11     1148        12                  Vine St and Opp Vineland Ter
12     1149        13                      

#### Plot the shortest route on map

In [361]:
gtfs_map = fl.Map(location = [stops_df['stop_lat'].median(), stops_df['stop_lon'].median()],
                  tiles = 'Stamen Toner', zoom_start = 15)
stop_clusters = MarkerCluster()
points = []

for row in stops_df.itertuples():
    point = (row.stop_lat, row.stop_lon)
    points.append(point)
    stop_clusters.add_child(fl.Marker(location = [row.stop_lat, row.stop_lon], popup = row.stop_name))

gtfs_map.add_child(stop_clusters)
gtfs_map.add_child(fl.PolyLine(points, color = "red", weight = 5, opacity = 0.5))